<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/Parseo_de_OrdenesdelDia_ListadoCerrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f "Scraps PDFs.csv"
!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f "base_formateada_senado_snowball.csv"

Mounted at /content/drive
100% 234M/234M [00:03<00:00, 84.8MB/s]
100% 234M/234M [00:03<00:00, 66.1MB/s]
 99% 141M/143M [00:01<00:00, 90.0MB/s]
100% 143M/143M [00:01<00:00, 81.4MB/s]


In [2]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [3]:
# Cargamos la base
base = pd.read_csv('/content/Scraps PDFs.csv')
base_parseada = pd.read_csv('/content/base_formateada_senado_snowball.csv')

<ipython-input-3-e0eeee1141a8>:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  base_parseada = pd.read_csv('/content/base_formateada_senado_snowball.csv')


In [4]:
base_parseada.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens'],
      dtype='object')

In [5]:
base_parseada.Link[30]

'https://www.senado.es/legis10/publicaciones/pdf/senado/ds/DS_P_10_2.PDF'

In [74]:
def eliminar_despues_de_autor(cadena):
    # Buscar la posición de la palabra "Autor" (sin importar mayúsculas o minúsculas)
    posicion_autor = cadena.lower().find('autor')

    # Si se encuentra la palabra "Autor"
    if posicion_autor != -1:
        # Eliminar todo lo que está después de la palabra "Autor"
        cadena = cadena[:posicion_autor]
    return cadena

In [109]:
def unir_strings_con_espacio(lista_strings):
    nueva_lista = []
    elemento_actual = ""

    for elemento in lista_strings:
        # Verificar si el elemento actual termina con un espacio
        if elemento_actual.endswith(' '):
            # Concatenar el elemento actual con el siguiente
            elemento_actual = elemento_actual + elemento
        else:
            # Si no termina con espacio, agregar el elemento actual a la nueva lista
            if elemento_actual:
                nueva_lista.append(elemento_actual)
            elemento_actual = elemento

    # Agregar el último elemento a la nueva lista
    if elemento_actual:
        nueva_lista.append(elemento_actual)

    return nueva_lista

In [123]:
def find_last_occurrence(text, substring):
    last_occurrence = -1
    index = text.find(substring)

    while index != -1:
        last_occurrence = index
        index = text.find(substring, index + 1)

    return last_occurrence

In [151]:
import re

def generar(l):
  df=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens', 'orden'])
  fila=base['link'][base['link'] == l[6]].index[0]
  texto=str(base.loc[fila].text)
  discurso=str(l[3])

  #ordenes = texto[texto.find('ORDEN DEL DÍA'):texto.find('SUMARIO')].split('\n')
  #ordenes=['JURAMENTO O PROMESA DE ACATAMIENTO A LA CONSTITUCIÓN.','1. ACTAS', '2. REGLAMENTO DEL SENADO', '3. INCOMPATIBILIDADES', '4. COMISIONES MIXTAS', '5. CONVENIOS Y ACUERDOS DE COOPERACIÓN ENTRE COMUNIDADES AUTÓNOMAS']
  #patron = r'\n\n([\s\S]*?)\n\n'
  #ordenes = re.findall(patron, texto[texto.find('ORDEN DEL DÍA'):texto.find('SUMARIO')])
  #ordenes=texto[texto.find('ORDEN DEL DÍA'):texto.find('SUMARIO')].split('\n')
  orden_del_dia_pattern = re.compile(r'\n(.*?)(?=\n)', re.DOTALL)
  matches = orden_del_dia_pattern.findall(texto[texto.find('ORDEN DEL DÍA'):texto.find('SUMARIO')])
  adjusted_matches = []
  for i, match in enumerate(matches):
      # Eliminar espacio inicial si existe
      match = match.lstrip()

      # Si el match comienza con minúscula, hacer join con el match anterior
      if match and match[0].islower() or match[:5]=='Autor' and i > 0:
          adjusted_matches[-1] += '' + match
      # Si el match comienza con '(', eliminar el match de la lista
      elif match and match[0] == '(' or match=='Pleno' or match[:3]=='Núm':
          continue
      else:
          adjusted_matches.append(match)

  ordenes=adjusted_matches

  o=0
  ubic_anterior=0

  ordenesnuevo=[]
  for orden in ordenes:
    if orden != 'JUNTA PREPARATORIA Y SESIÓN CONSTITUTIVA':
      orden=str(eliminar_despues_de_autor(orden))
      ordenesnuevo.append(orden.replace('— ','').replace('  ',' '))

  ordenes=ordenesnuevo
  ordenes=unir_strings_con_espacio(ordenes)
  #print(ordenes)

  for orden in ordenes:
    #print(texto[texto.find('SUMARIO'):].replace('\n','').replace('\t', '').replace(' ','').find(l[2].replace('\n','').replace('\t', '').replace(' ','')+':'+ discurso.replace('\t', '').replace('\n','').replace(' ','')))
    #print(texto[texto.find('SUMARIO'):].replace('\n','').replace('\t', '').replace(' ','').find(orden.upper().replace('\n','').replace('\t', '').replace(' ','')))
    #print(texto[texto.find('SUMARIO'):].replace('\n','').replace('\t', '').replace(' ',''))
    #print(l[2].replace('\n','').replace('\t', '').replace(' ','')+':'+discurso.replace('\n','').replace('\t', '').replace(' ',''))

    texto_enelquebuscar=texto[texto.find('SUMARIO'):].replace('\n','').replace('\t','').replace(' ','')
    discurso_con_nombre=str(l[2]+': '+l[3]).replace('\n','').replace('\t','').replace(' ','')
    ubic_discurso=texto_enelquebuscar.find(discurso_con_nombre)
    #ubic_titulo=texto_enelquebuscar.find(orden.upper().replace('\n','').replace('\t', '').replace(' ','')[3:])
    ubic_titulo=find_last_occurrence(texto_enelquebuscar, orden.upper().replace('\n','').replace('\t', '').replace(' ','')[3:])
    if ubic_titulo==-1: ubic_titulo=ubic_anterior+1
    ubic_anterior=ubic_titulo
    #print(ubic_titulo)
    #print(ubic_discurso)
    #print('----------')
    if ubic_discurso>ubic_titulo: o+=1


  #print(texto_enelquebuscar)
  #print(discurso_con_nombre)
  df['Nombre/Cargo']=[l[2]]
  df['Discurso']=[l[3]]
  df['Cargo']=[l[4]]
  df['Nombre']=[l[5]]
  df['Link']=[l[6]]
  df['Legislatura']=[l[7]]
  df['tipo']=[l[8]]
  df['fecha']=[l[9]]
  df['fecha_formato']=[l[10]]
  df['nombre']=[l[11]]
  df['apellidos']=[l[12]]
  df['procedLiteral']=[l[13]]
  df['procedLugar']=[l[14]]
  df['grupoNombre']=[l[15]]
  df['tokens']=[l[16]]
  try:
    df['orden']=[ordenes[o-1].upper()]
  except:
    df['orden']=[""]


  return df

In [152]:
base2=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens', 'orden'])

In [153]:
for row in base_parseada.values[307:308]:
  base2 = pd.concat([base2, generar(row)], axis=0)

In [149]:
base2

,Nombre/Cargo,Discurso,Cargo,Nombre,Link,Legislatura,tipo,fecha,fecha_formato,nombre,apellidos,procedLiteral,procedLugar,grupoNombre,tokens,orden
0,El señor FERNÁNDEZ ORTEGA,"Termino, señor presidente. …y la participación...",FERNÁNDEZ ORTEGA,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENO,7 de marzo de 2012,2012-03-07,JUAN MANUEL,FERNÁNDEZ ORTEGA,Electo,Electo: Granada,GPS,"['termin', '…y', 'particip', 'institu', 'socie...",5.1. MOCIÓN POR LA QUE SE INSTA AL GOBIERNO A ...


In [154]:
base2.Link[0]

'https://www.senado.es/legis10/publicaciones/pdf/senado/ds/DS_P_10_8.PDF'

In [ ]:
base2=base2.reset_index(drop=True)

In [ ]:
base2['fecha_formato'] = pd.to_datetime(base2['fecha_formato'], errors='coerce')

In [ ]:
from google.colab import files
base2.to_csv('base_formateada_senado_snowball_conorden.csv', escapechar='\\')
files.download('base_formateada_senado_snowball_conorden.csv')